In [8]:
#Get pipeline run  info
from kfp import client

# Create a KFP client
kfp_client = client.Client()

# List all runs (you can specify the number of runs)
runs = kfp_client.list_runs(page_size=5)
if runs is None or runs.runs is None:
    print("No runs found")
# Print the ID and details of each run
else:
    for run in runs.runs:
        print(f"Pipeline Run ID: {run.run_id},  Name: {run.display_name}, Status: {run.state}")


Pipeline Run ID: 76e9de7d-9acb-40c5-acff-c6435aead7e2,  Name: chain_shipments_anonymize_chain_run, Status: SUCCEEDED
Pipeline Run ID: 71ee997c-08f4-4731-a935-b14bcef8a9d7,  Name: chain_shipments_data_chain_run, Status: SUCCEEDED


In [131]:
# delete a specific pipline

from kfp import client

# Create a KFP client
kfp_client = client.Client()

# Replace this with your pipeline ID
pipeline_id = '<your_pipeline_id>'

# Delete the pipeline
kfp_client.delete_pipeline(pipeline_id)

print(f"Pipeline with ID '{pipeline_id}' has been deleted.")


ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Audit-Id': '11d78cd7-dbb2-4ccf-94eb-2a98d751218a', 'Cache-Control': 'no-cache, private', 'Content-Length': '623', 'Content-Type': 'application/json', 'Date': 'Fri, 01 Nov 2024 13:01:02 GMT'})
HTTP response body: {"error":"Failed to delete pipeline \u003cyour_pipeline_id\u003e. Check error stack: Failed to delete pipeline with id \u003cyour_pipeline_id\u003e as it was not found: ResourceNotFoundError: Pipeline \u003cyour_pipeline_id\u003e not found","code":5,"message":"Failed to delete pipeline \u003cyour_pipeline_id\u003e. Check error stack: Failed to delete pipeline with id \u003cyour_pipeline_id\u003e as it was not found: ResourceNotFoundError: Pipeline \u003cyour_pipeline_id\u003e not found","details":[{"@type":"type.googleapis.com/google.rpc.Status","code":5,"message":"Pipeline \u003cyour_pipeline_id\u003e not found"}]}


In [28]:
# Delete all pipelines

from kfp import Client

# Create a KFP client
kfp_client = Client()

# List all pipelines
pipelines = kfp_client.list_pipelines()

# Delete each pipeline and its versions
for pipeline in pipelines.pipelines:
    # Get all versions of the pipeline
    versions = kfp_client.list_pipeline_versions(pipeline_id=pipeline.pipeline_id)
    
    # Delete each pipeline version
    for version in versions.pipeline_versions:
        kfp_client.delete_pipeline_version(pipeline.pipeline_id,version.pipeline_version_id)
        print(f"Deleted version: {version.display_name} (ID: {version.pipeline_version_id}) of pipeline: {pipeline.display_name}")
    
    # Now delete the pipeline itself
    kfp_client.delete_pipeline(pipeline.pipeline_id)
    print(f"Deleted pipeline: {pipeline.display_name} (ID: {pipeline.pipeline_id})")



Deleted version: chain_employee_data_pipeline_618e4a19-530b-405b-9ce6-a1f7788b9778 (ID: 533b6624-c12f-4729-b780-df41095acb4d) of pipeline: chain_employee_data_pipeline_618e4a19-530b-405b-9ce6-a1f7788b9778
Deleted pipeline: chain_employee_data_pipeline_618e4a19-530b-405b-9ce6-a1f7788b9778 (ID: 5f76707a-16f1-433c-bd0a-5a70795bbc72)
Deleted version: chain_sales_data_pipeline_764bd130-0123-4a60-9f4e-31ff2b14b47f (ID: d6e2baf8-bc88-4c41-9836-86a2624f5b4d) of pipeline: chain_sales_data_pipeline_764bd130-0123-4a60-9f4e-31ff2b14b47f
Deleted pipeline: chain_sales_data_pipeline_764bd130-0123-4a60-9f4e-31ff2b14b47f (ID: 691eb105-cafc-4367-8043-59cf1608b737)


In [31]:
# Delete All the experiments that their runs in state ['CANCELED', 'FAILED', 'SKIPPED','SUCCEEDED']

import kfp

# Connect to the Kubeflow Pipelines API
client = kfp.Client()

# List all experiments
experiments = client.list_experiments()

# Iterate over all experiments
if experiments is not None or experiments.experiments is not None:
    for experiment in experiments.experiments:
        print(f"Checking Experiment: {experiment.display_name} (ID: {experiment.experiment_id})")
        
        # List all runs for the current experiment
        runs = client.list_runs(experiment_id=experiment.experiment_id)
        
        all_runs_completed = True  # Flag to track if all runs are completed
        
        if runs and runs.runs:
            for run in runs.runs:
                print(f"  Run Name: {run.display_name}, Run ID: {run.run_id}")
                
                # # Check if the run is not completed
                if run.state not in ['CANCELED', 'FAILED', 'SKIPPED','SUCCEEDED']:
                    print(f"  Run {run.display_name} is not completed. Skipping deletion of this experiment.")
                    all_runs_completed = False
                    break
        
        # If all runs are completed, delete the experiment and its runs
        if all_runs_completed:
            print(f"  All runs for experiment {experiment.display_name} are completed. Deleting runs and experiment.")
            
            # Delete all runs for this experiment
            if runs and runs.runs:
                for run in runs.runs:
                    print(f"    Deleting run: {run.display_name}")
                    client.delete_run(run_id=run.run_id)
                
                # Delete the experiment
                client.delete_experiment(experiment_id=experiment.experiment_id)
                print(f"    Experiment {experiment.display_name} deleted.")
        else:
            print(f"  Skipping deletion of experiment {experiment.display_name} as some runs are still in progress.")
    
print("Completed cleanup of experiments.")


Checking Experiment: Default (ID: b9a875a8-fb4f-4974-9be5-e1a547aaedf6)
  All runs for experiment Default are completed. Deleting runs and experiment.
Checking Experiment: pipeline_employee_data_experiment (ID: fa55e32b-423d-447b-b81f-fc8734bf21ee)
  All runs for experiment pipeline_employee_data_experiment are completed. Deleting runs and experiment.
Completed cleanup of experiments.


In [2]:
# Get list of pipelines

from kfp import client

# Create a KFP client
kfp_client = client.Client()

# List all pipelines
pipelines = kfp_client.list_pipelines()
if pipelines is None or pipelines.pipelines is None:
    print("No pipeline found")
else:   
    for pipeline in pipelines.pipelines:
        print(f"ID: {pipeline.pipeline_id} ,Name: {pipeline.display_name}")
        print(pipeline)

ID: 7a28e47b-4d41-46df-afa6-4f1bbd06e591 ,Name: chain_shipments_anonymize_chain
{'created_at': datetime.datetime(2024, 12, 13, 13, 16, 5, tzinfo=tzutc()),
 'description': None,
 'display_name': 'chain_shipments_anonymize_chain',
 'error': None,
 'namespace': None,
 'pipeline_id': '7a28e47b-4d41-46df-afa6-4f1bbd06e591'}
ID: d8f13d9c-753f-420b-ab27-c39a0be1dbe4 ,Name: chain_shipments_data_chain
{'created_at': datetime.datetime(2024, 12, 13, 13, 16, 5, tzinfo=tzutc()),
 'description': None,
 'display_name': 'chain_shipments_data_chain',
 'error': None,
 'namespace': None,
 'pipeline_id': 'd8f13d9c-753f-420b-ab27-c39a0be1dbe4'}


In [5]:
#pipeline versions

from kfp import client

# Create a KFP client
kfp_client = client.Client()

# Replace with your actual pipeline ID
pipeline_id = 'd8f13d9c-753f-420b-ab27-c39a0be1dbe4'

# List pipeline versions
pipeline_versions = kfp_client.list_pipeline_versions(pipeline_id=pipeline_id)

# Print version IDs and names
for version in pipeline_versions.pipeline_versions:
    print(f"Version Name: {version.display_name}, Version Pipeline ID: {version.pipeline_version_id}")


Version Name: chain_shipments_data_chain, Version Pipeline ID: 1657e6c1-28e7-49e0-893a-73de33ccb749


In [7]:
#create a running experiment from pipelines
# from kfp import client
# 
# # Create a KFP client
# kfp_client = client.Client()
# 
# # Define your pipeline function and experiment details
# experiment_name = 'my-experiment'
# run_name = 'my-pipeline-run'
# 
# # Submit the pipeline run
# run = kfp_client.create_run_from_pipeline_func(
#     pipeline_func=my_pipeline_func,
#     arguments={},
#     experiment_name=experiment_name,
#     run_name=run_name
# )
# 
# # Get the run ID
# pipeline_run_id = run.run_id
# print(f"Pipeline Run ID: {pipeline_run_id}")

from kfp import client

# Create a KFP client
kfp_client = client.Client()
#set up the Minio 
MINIO_ENDPOINT = 'http://minio.kubeflow:9000'  # Update with your MinIO service endpoint
BUCKET_NAME = 'my-bucket'  # Replace with your desired bucket name
ACCESS_KEY = 'minio'  # Replace with your access key
SECRET_KEY = 'minio123'  # Replace with your secret key
DEFAULT_REGION ='us-east-1'
# Variables (replace with your values)
pipeline_name = "chain_shipments_data_chain"  # Name of the pipeline after upload
experiment_name = "chain_shipments_data_chain_experiment"  # The experiment you want to run the pipeline in
run_name = "chain_shipments_data_chain_run"  # A name for the run you're about to create
arguments = {
            # 'storage_type': 'S3',
            # 's3_bucket': BUCKET_NAME,
            # 's3_endpoint': MINIO_ENDPOINT,
            # 's3_access_key': ACCESS_KEY,
            # 's3_secret_key': SECRET_KEY,
            # 's3_secure': 'false'  # Set to 'true' if using HTTPS
        }
pipeline_root = f"s3://my-bucket/kubeflow/pipelines"
version_id ='1657e6c1-28e7-49e0-893a-73de33ccb749'

# Step 1: Upload the pipeline (already done in your case)
# pipeline_file_name = 'path_to_your_pipeline.yaml'
# kfp_client.upload_pipeline(pipeline_package_path=pipeline_file_name, pipeline_name=pipeline_name)

# Step 2: Get the pipeline ID (this is the pipeline you previously uploaded)
pipelines = kfp_client.list_pipelines()
pipeline_id = None
for pipeline in pipelines.pipelines:
    if pipeline.display_name == pipeline_name:
        pipeline_id = pipeline.pipeline_id
        print(f"Pipeline ID: {pipeline_id}")
        break

if pipeline_id is None:
    raise ValueError(f"Pipeline '{pipeline_name}' not found. Please check the pipeline name.")

# Step 3: Get or create the experiment
# Check if the experiment already exists
experiments = kfp_client.list_experiments()
experiment_id = None
for experiment in experiments.experiments:
    if experiment.display_name == experiment_name:
        experiment_id = experiment.experiment_id
        print(f"Experiment ID: {experiment_id}")
        break

# If the experiment doesn't exist, create a new one
if experiment_id is None:
    experiment = kfp_client.create_experiment(name=experiment_name)
    experiment_id = experiment.experiment_id
    print(f"New Experiment ID: {experiment_id}")
    
#    

# Step 4: Run the pipeline using the pipeline ID and experiment
run = kfp_client.run_pipeline(
    experiment_id=experiment_id,
    job_name=run_name,
    pipeline_id=pipeline_id,
    params=arguments,
    version_id=version_id #,
   # pipeline_root=pipeline_root
)

print(f"Run submitted. Run ID: {run.run_id}")



Pipeline ID: d8f13d9c-753f-420b-ab27-c39a0be1dbe4


New Experiment ID: 1b69bf2a-f7f9-4644-9fed-027b1f31ea5c


Run submitted. Run ID: 71ee997c-08f4-4731-a935-b14bcef8a9d7
